<b style='Color:red'> The python version used for running this notebook is 3.11.8 </b>

In [ ]:
import matplotlib.pyplot as plt
import h5py as h5
import numpy as np
import cv2, math
from skimage import morphology,filters, util
import seaborn as sns
import pandas as pd
import os

In [2]:
# Please replace it with the correct file path on your computer
filePath = "C:/Users/user/Documents/BHE/BRUNNER/data_v3/SCORPIO_LWIR/"
filePath = "C:/Users/user/Documents/BHE/BRUNNER/data_v1/"


# Create a directory for saving results

In [3]:
# set directory for plots
if not(os.path.isdir("./results")):
    os.mkdir("./results")

# Gaussian Filtration with Heatmap

What we need to do next is to find a way to classify columns chronologically. I had some ideas but still need some fine-tuning.

In [4]:
filename = "REF_N5.h5"
file = h5.File("{}{}".format(filePath,filename), 'r')
df_total = pd.DataFrame()
testImages = file.keys()
T1 = 7 #We deduced this value experimentally (tested on different data samples).
for k in testImages:
    imagename = k
    image1 = file[imagename]
    image1 = file[imagename][:image1.shape[1]]

    image_uint8 = util.img_as_ubyte(image1 / np.max(image1))
    footprint = morphology.footprint_rectangle((30, 1))
    # filter using mean bilateral method
    filtered = filters.rank.mean_bilateral(image_uint8, footprint, s0=10, s1=10)

    # Apply thresholding
    _, thresholded = cv2.threshold(filtered, 125, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C)  # in some dataset, we needed to lower the min value to 60 instead of 125
    #if the image has white background with black noise after cleaning, inverse it
    if np.mean(thresholded) > 200:
        thresholded = abs(255 - thresholded)
    window_height = 30
    h, w = thresholded.shape

    # Normalize thresholded image to binary (0 or 1)
    binary = (thresholded > 0).astype(np.uint8)
    binary2 = (thresholded>0)
    num_windows = math.ceil(h / window_height)
    column_window_sums = np.zeros((num_windows, w), dtype=np.uint16)

    for win in range(num_windows):
        start_row = win * window_height
        end_row = start_row + window_height
        window_slice = binary[start_row:end_row, :]
        column_window_sums[win,:] = window_slice.sum(axis=0)
    column_window_sums[column_window_sums<T1] = 0

    defect_columns = np.where(column_window_sums >= T1)[1]
    affected_windows_per_column = {}
    for col in defect_columns:
        count = np.count_nonzero(column_window_sums[:, col])
        #if the column has only 1 window with lowe value, remove it
        if count == 1 and column_window_sums[:, col].sum() <13:
            column_window_sums[:, col] = 0
        else:
            lbl = int(col)
            affected_windows_per_column[lbl] = count
    #if the frame has any detected faulty column, create plots and append the columns to data frame
    if affected_windows_per_column:
        fig,ax=plt.subplots(2,2,figsize=(16, 9))

        ax[0][0].set_title("Original Image")
        ax[0][0].imshow(image_uint8,cmap='gray')
        fig.suptitle("Local Dark Pixel Density {}".format(k))
        ax[0][1].imshow(thresholded,cmap='gray')
        ax[0][1].set_title("Thresholded and smoothed Image")
        # if affected_windows_per_column:
        plt1=ax[1][0].imshow(column_window_sums, cmap='hot', aspect='auto')
        cbar = fig.colorbar(plt1, ax=ax[1][0])
        cbar.set_label("White Spot Count in 30-px Window")
        ax[1][0].set_title("Column HeatMap")
        fig.delaxes(ax[1][1])
        fig.savefig('./results/{}.png'.format(k))
        plt.close()
        data_dict = {}
        for col in affected_windows_per_column.keys():
            data_dict[col] = column_window_sums[:, col]

        # Create DataFrame from dict
        df = pd.DataFrame(data_dict)
        df['window'] = df.index  # Add window index for x-axis

        # Melt for grouped bar chart
        df_melted = df.melt(id_vars='window', var_name='column', value_name='value')
        df_melted['frame'] = k
        df_melted['windows_total'] = num_windows
        df_total = pd.concat([df_total,df_melted])




In [8]:
dict_analysis = {"frame":[],'column':[],'windows':[],"values_mean":[],"values_sd":[],"type":[],"start":[],"end":[]}
#types can be F:fractioned, C:complete,
df_c = df_total.copy()
df_c = df_c[df_c['value'] >0]
frames = list(df_c['frame'].unique())
for frame in frames:
    df_f = df_c.copy()
    df_f = df_f[df_f['frame'] == frame]
    columns = list(df_f['column'].unique())
    type = ""
    for column in columns:
        df_col = df_f.copy()
        df_col = df_col[df_col['column'] == column]
        nb_windows = df_f.shape[0]
        if nb_windows == 1:
            min_index = df_col['window'].iloc[0]
            max_index = min_index
            type = "Partial"
        else:
            min_index = df_col['window'].min()
            max_index = df_col['window'].max()
            total_windows = df_col['windows_total'].iloc[0]
            if (max_index - min_index)/total_windows >=0.7:
                type = 'Complete'
            else:
                type = 'Partial'
    dict_analysis['frame'].append(frame)
    dict_analysis['column'].append(column)
    dict_analysis['windows'].append(nb_windows)
    dict_analysis['values_mean'].append(df_col['value'].mean())
    dict_analysis['values_sd'].append(df_col['value'].std())
    dict_analysis['start'].append(min_index)
    dict_analysis['end'].append(max_index)
    dict_analysis['type'].append(type)

In [15]:
df_analysis = pd.DataFrame(dict_analysis)

In [ ]:
df_analysis['frame'] = df_analysis['frame'].apply(lambda x: int(x.replace("Image ","")))
df_analysis = df_analysis.sort_values(['column','frame']).reset_index(drop=True)
cnstColThreshold = 3
def assign_groups(group):
    group = group.copy()
    group["frame_diff"] = group["frame"].diff().fillna(1)
    group["chunk"] = (group["frame_diff"] > 1).cumsum()
    return group

df_analysis = df_analysis.groupby("column", group_keys=False).apply(assign_groups)
#if the same column is noisy in more that 2 consecutive frames, it is constant
def label_status(subgroup):
    frames = subgroup["frame"].values
    status = "temporal"
    for i in range(len(frames) - 2):
        if frames[i+1] == frames[i] + 1 and frames[i+2] == frames[i] + 2:
            status = "constant"
            break
    subgroup["status"] = status
    return subgroup

df_analysis = df_analysis.groupby(["column", "chunk"], group_keys=False).apply(label_status)
df_analysis = df_analysis.drop(columns=["frame_diff"])
df_analysis = df_analysis.drop(columns=["chunk"])
df_analysis.to_csv("./results/{}.csv".format(filename.replace(".h5","")))

C:\Users\user\AppData\Local\Temp\ipykernel_37456\1940942491.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_analysis = df_analysis.groupby("column", group_keys=False).apply(assign_groups)
C:\Users\user\AppData\Local\Temp\ipykernel_37456\1940942491.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_analysis = df_analysis.groupby(["column", "chunk"], group_keys=False).apply(label_status)
